# Using PBP + NBA Traditional BoxScore Data to estimate missed games

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/pbp_related/"
img_DIR_P = "../data/images/players/"
img_DIR_T = "../data/images/teams/"
data_DIR = "../data/pbpdata/"
box_DIR = "../data/box/"
aio_DIR = "../data/all_in_one_metrics/"

In [ ]:
df0 = get_box("P","Trad",False,[2024])
df0.columns
df0 = df0[['gameid', 'personid', 'comment']]
df0.columns = ['game_id', 'player_id', 'comment']

In [ ]:
league = "NBA"
season = "2024"
year = season + "-" + str(int(season)+1)[-2:]
with zstd.open(data_DIR + league+"_PBPdata_" + season + ".pkl.zst","rb") as f:
    games_list = dill.load(f)
player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()
len(games_list)

In [ ]:
dfa = []
for game in games_list:
    df1 = pd.DataFrame(game.boxscore.player_items)
    df1["game_id"] = game.game_id
    dfa.append(df1)
df2 = pd.concat(dfa)

In [ ]:
cols = ['game_id','team_id', 'team_abbreviation', 'player_id','name', 'status', 'totsec']
df3 = df2[cols]
df3["game_id"] = df3["game_id"].astype(int)
df3 = df3.rename(columns={"name":"Name"}).reset_index(drop=True)
# dfm1 = df3.query("status == 'I'").reset_index(drop=True)
dfm1 = df3.query("totsec == 0").reset_index(drop=True)
dfm1 = pd.merge(dfm1,df0,how="left")
dfm1 = dfm1[~dfm1["comment"].isin(["DNP - Coach's Decision","DND - Coach's Decision"])]
dfm1 = dfm1.reset_index(drop=True)

In [ ]:
dflb0 = pd.read_csv(aio_DIR + "NBA_LEBRON_FULL.csv")

In [ ]:
cols2 = ["player_id","player_name","Age","G","Mins","MPG","LEBRON","LEBRON_WinsAdded","multiLEBRON","multiLEBRON_WAR_Projection"]
dflb = dflb0.query("Year == 2025")
dflb = dflb[cols2]
dflb = dflb.dropna()
dflb["Min"] = dflb["MPG"]
dflb["LBWAR_PG"] = dflb["multiLEBRON_WAR_Projection"]/82
# dflb["LBWAR_PG"] = dflb["LEBRON_WinsAdded"]/dflb["G"]
dflb.iloc[:,3:-1] = round(dflb.iloc[:,3:-1],4)
dflb["LBWAR_PG"] = dflb["LBWAR_PG"].round(5)
dflm = dflb[["player_id","LBWAR_PG","Min"]]

In [ ]:
# print(dflb.query("player_name == 'Oscar Tshiebwe'").to_markdown())

In [ ]:
dflb2 = dflb0.query("Year == 2024")
dflb2 = dflb2[cols2]
dflb2["Min"] = dflb2["MPG"]
dflb2["LBWAR_PG"] = dflb2["multiLEBRON_WAR_Projection"]/82
# dflb2["LBWAR_PG"] = dflb2["LEBRON_WinsAdded"]/dflb2["G"]
dflb2.iloc[:,3:-1] = round(dflb2.iloc[:,3:-1],4)
dflb2["LBWAR_PG"] = dflb2["LBWAR_PG"].round(5)
dflm2 = dflb2[["player_id","LBWAR_PG","Min"]]

In [ ]:
dfm2 = pd.merge(dfm1,dflm,on="player_id",how="left")
dfm3 = dfm1[dfm2["LBWAR_PG"].isna()]
dfm4 = pd.merge(dfm1,dflm,on="player_id")
dfm5 = pd.merge(dfm3,dflm2,on="player_id")
dfm = pd.concat([dfm4,dfm5])
dfm = dfm[~dfm.duplicated(keep="last")].reset_index(drop=True)

In [ ]:
len(dfm)-len(dfm1)

In [ ]:
df10 = pd.read_parquet(box_DIR + f"NBA_BOX_T_Base_{season}.parquet")
df10= df10[["GAME_ID","TEAM_ID","GAME_DATE","MATCHUP","WL","PLUS_MINUS"]]
df10 = df10.rename(columns={"GAME_ID":"game_id","TEAM_ID":"team_id"})
df10["game_id"] = df10["game_id"].astype(int)

In [ ]:
team = "MIL"
dfmt =  (
    dfm
    .query(f"team_abbreviation == '{team}'")
    .groupby(["Name"])[["Name","LBWAR_PG","Min"]]
    .agg({"Name":["count"],"Min":["sum"],"LBWAR_PG":["sum"]})
)
dfmt.columns = ["Games","Min","WAR"]
dfmt = dfmt.sort_values("WAR",ascending=False).reset_index()
dfmt["Min"] = dfmt["Min"].round()
dfmt["WAR"] = dfmt["WAR"].round(3)
dfmt

In [ ]:
dfmg =  (
    dfm
    .groupby(["game_id","team_id"])[["player_id","LBWAR_PG","Min"]]
    .agg({"player_id":["count"],"Min":["sum"],"LBWAR_PG":["sum"]})
)
dfmg.columns = ["Games_Missed","Minutes_Missed","LEBRON_WAR_Missed"]
dfmg = dfmg.reset_index()
# merge team boxscores to get game details like date and matchup
dfmf = pd.merge(df10,dfmg,on=["game_id","team_id"])
dfmf["Team"] = dfmf["team_id"].map(teams_dict)
dfmf = dfmf.drop(columns=["team_id","team_id"])
dfmf.insert(1,"Team",dfmf.pop("Team"))
df_x = dfmf.groupby("Team").agg({"Games_Missed":["sum"],"Minutes_Missed":["sum"],"LEBRON_WAR_Missed":["sum"]})
df_x.columns = ["Games_Missed","Minutes_Missed","LEBRON_WAR_Missed"]
df_x = (df_x
        .reset_index()
        .sort_values("LEBRON_WAR_Missed",ascending=False)
        .reset_index(drop=True)
    )
df_x["LEBRON_WAR_Missed"] = df_x["LEBRON_WAR_Missed"].round(2)
df_x["Minutes_Missed"] = df_x["Minutes_Missed"].astype(int)
df_x["logo"]= df_x["Team"]
df_x.insert(0,"logo",df_x.pop("logo"))

In [ ]:
data = df_x.reset_index()
data["index"] +=1
t = (
        GT(data)
        .tab_header(
            title = md("**NBA Wins Missed Leaderboard 2024-25**"),
            subtitle = md("Includes games player missed due to Injuries, suspensions, rest, personal issues etc. (except for DNP CD)<br />Wins missed calculated using **multi LEBRON WAR**. If the player didn't play this season, last season's value is used")  
        )
        .tab_source_note(source_note="socials:@sradjoker | source: bball-index, pbpstats")
        .cols_label(
            index = "#",
            logo = "",
            Games_Missed = "Games Missed",
            Minutes_Missed = "Minutes Missed",
            LEBRON_WAR_Missed="Wins Missed"
        )
        .data_color(
            columns=['Games_Missed','Minutes_Missed','LEBRON_WAR_Missed'],
            palette="RdYlGn",
            reverse="True"
        )
        .fmt_image(
            columns="logo",
            path = img_DIR_T,
            file_pattern="{}.png"
        )
        .cols_align(align="center")
        .cols_align(align="left", columns="Team")
        .tab_options(
            heading_title_font_size="200%",
            heading_subtitle_font_size="100%",
            # heading_title_font_weight='bold',
            table_background_color="floralwhite",
            column_labels_font_size="105%",
            column_labels_font_weight='bold',
            row_group_font_weight='bold',
            row_group_background_color="#E5E1D8",
            table_font_size=10,
            table_font_names="Consolas", 
            data_row_padding = "0.5px",
            # table_margin_left = 7,
            # table_margin_right = 0,
        )
    )
t.save(fig_DIR + f"NBA_mWAR_Missed_{season}.png",scale=3,web_driver="firefox")
t

In [ ]:
df_teams = pd.read_csv("../data/" + "NBA_teams_colors_logos.csv")
df_teams = df_teams[["Team","colorsTeam"]]
df_y = pd.merge(df_x, df_teams, on="Team")

In [ ]:
df_avg = df_y.iloc[:,2:-1].mean()
df_avg = pd.DataFrame(df_avg).T
df_avg["Games_Missed"] = df_avg["Games_Missed"].round(0)
df_avg["Minutes_Missed"] = df_avg["Minutes_Missed"].round(1)
df_avg["LEBRON_WAR_Missed"] = df_avg["LEBRON_WAR_Missed"].round(2)
df_avg["Team"] = "League Average"
df_avg["colorsTeam"]  = "#000000"

In [ ]:
df_z = pd.concat([df_y,df_avg]).sort_values("LEBRON_WAR_Missed",ascending=False).reset_index(drop=True)
teams = df_z["Team"].to_list()
teams.reverse()
df_z["image"]= img_DIR_T + df_z["Team"] + ".png"
df_z["Team"] = pd.Categorical(df_z['Team'], categories=teams)
df = df_z
lb_mean = df["LEBRON_WAR_Missed"].mean()
lb_mean = round(lb_mean,2)
# df1 = df.copy()
# df1["LEBRON_WAR_Missed"] = df1["LEBRON_WAR_Missed"] - lb_mean
# df1["LEBRON_WAR_Missed"] = df1["LEBRON_WAR_Missed"].round(3)

In [ ]:
lb_mean

In [ ]:
p = (
    ggplot(df, aes(x='Team', y='LEBRON_WAR_Missed', image="image"))
    + geom_bar(aes(fill="colorsTeam"),stat="identity", alpha=0.7,show_legend=False)
    + coord_flip()
    + scale_color_identity(aesthetics=["fill"])
    + scale_y_continuous(breaks=np.arange(0,11,2))
    + geom_image(aes(y=-0.2), size=0.05)
    + geom_hline(yintercept=lb_mean, linetype="dashed")
    # + theme_538(base_size=12)
    # + theme_classic(base_size=12)
    + theme_idv
    + theme(
        figure_size=(10,10),
        plot_title=element_text(size=20, weight="bold"),
    )
    + labs(
        title="LEBRON WAR Missed as of "+ datetime.today().strftime("%Y-%m-%d"),
        subtitle=f"Players missing the game due to being Inactive",
        caption="bsky:@sradjoker.cc | X:@sradjoker | Source: pbpstats, bball-index",
        y= "LEBRON WAR Missed",
        x="",
    )
)
p.save(fig_DIR + f"mLEBRON_WAR_Missed_{season}",dpi=300,verbose=False)
p